# **Design for Single Differenced Kalman Network for GNSS Triangulation**

### **Introduction**

Here, we design a Recurrent Neural Network (RNN) aided kalman filter for GPS/GNSS triangulation. The main motivation behind this is that this is able to learn the non-linear filter for a given reciever and satellite constellation and the reciever dependent noise characteristics. 

The main paper that we are following:

- [KalmanNet: Neural Network Aided Kalman Filtering for Partially Known Dynamics](https://ieeexplore.ieee.org/abstract/document/9733186?casa_token=E7Dlt41RuYMAAAAA:K_hc0gJoRDXLG3BNt6tS3QW1Tzf54jufij9adE7HcjGst0VeyXWhWe5Mc1MiyXH7tY_gcH0GNg)


This architecture has shown performance improvements over the traditional Kalman Filter (KF), Extended Kalman Filter (EKF), and Unscented Kalman Filter (UKF) in the case of non-linear dynamics and unknown noise characteristics as well as unknown dynamics. The main idea is to use a RNN to learn the dynamics and the noise characteristics of the system. Rather than tracking second order statistics of the system, we let the RNN learn the second order statistics of the system.  

## **Problem Formulation**

The GPS/GNSS problem is a non-linear problem. The measurement model in the non-linear observation equations. The dynamics, for example, the position of the moving object, is also generally non-linear. Hence, the traditional KF is not applicable. The EKF and UKF are applicable, but they require the knowledge of the noise characteristics of the system. This is not always available or easy to obtain. However, the RNN can learn the noise characteristics of the given system using the labeled data.

The state space model for the GPS/GNSS problem is genrally given as:
$$
x = \begin{bmatrix} x \\ y \\ z \\ \dot{x} \\ \dot{y} \\ \dot{z} \\ cdt \\ c\dot{dt}\end{bmatrix}
$$
where $x,y,z$ are the position of the reciever in the ECEF frame, $\dot{x},\dot{y},\dot{z}$ are the velocity of the reciever in the ECEF frame, $cdt$ is the clock bias of the reciever, and $c\dot{dt}$ is the clock drift of the reciever with respect to the GPS/GNSS time.

The pseudorange equation is given as:
$$
\rho = \sqrt{(x-x_s)^2 + (y-y_s)^2 + (z-z_s)^2} + cdt + \epsilon
$$
where $x_s,y_s,z_s$ are the position of the satellite in the ECEF frame, and $\epsilon$ is the noise in the measurement.

These were the assumption made in the [UKF Aided Kalman Filter](unscented_kalman_filter_gps.ipynb) notebook.

## **Architecture Design Problem** 

There are few design problems that we need to solve before we can implement the KalmanNet.

1. How does the training data look like?  
    Just like any neural network, training data will contaion an input X and a labelled output Y. Since the base architecture of the Kalman Net is RNN, the input will be time series of the observables,
    $$
    P_i = \begin{bmatrix} \rho_{1,i} \\ \rho_{2,i} \\ \vdots \\ \rho_{N_s,i} \end{bmatrix}
    $$
    where $i$ is the time index, and $N_s$ is the number of satellites in view. The labelled training data Y will be the time series of position and velocity of the reciever in the ECEF frame.
    $$
    Y_i = \begin{bmatrix} x_i \\ y_i \\ z_i \\ \dot{x}_i \\ \dot{y}_i \\ \dot{z}_i \end{bmatrix}
    $$

2. What about the reciever clock bias and drift?  
    The reciever clock bias and drift are not observables since they are very hard to measure with high accuracy. So, they are not included in the training data. So, we will have to eliminate them and gain some advantage from that. This is done by using the single differenced measurement model. The single differenced measurement mode. Hence out state space model becomes:
    $$
    x = \begin{bmatrix} x \\ y \\ z \\ \dot{x} \\ \dot{y} \\ \dot{z} \end{bmatrix}
    $$
    Hence, the clock bias is not calculated. This can be calculated using least squares or UKF. Eliminating the clock bias, we have more gain more precision in the location estimation and velocity estimation which are the main goals of the GPS/GNSS problem.


##

## **Single Differenced Equations and the measurement model**

To eliminate the reciever clock bias, we use the single differenced measurement model. This measurement model is given as:
$$
\rho_{ij} = \rho_{i} - \rho_{j} + \epsilon_{ij}
$$
where $\rho_{ij}$ is the single differenced pseudorange between the $i^{th}$ and $j^{th}$ reference satellite, $\rho_{i}$ is the pseudorange of the $i^{th}$ reference satellite, $\rho_{j}$ is the pseudorange of the $j^{th}$ reference satellite, and $\epsilon_{ij}$ is the remaining noise in the single differenced pseudorange.

Since the reciever clock bias is same for all the satellites at a given time, the pseudorange equation for the $i^{th}$ satellite is given as:
$$
\rho_{i} = \sqrt{(x-x_i)^2 + (y-y_i)^2 + (z-z_i)^2} + cdt + \epsilon_i
$$
where $x_i,y_i,z_i$ are the position of the $i^{th}$ satellite in the ECEF frame, and $\epsilon_i$ is the noise in the measurement. Now a reference satellite is chosen, generally having the highest elevation angle as the reference satellite. The single differenced pseudorange equation for the $i^{th}$ and $j^{th}$ satellite is given as:
$$
\rho_{ij} = \sqrt{(x-x_i)^2 + (y-y_i)^2 + (z-z_i)^2} - \sqrt{(x-x_j)^2 + (y-y_j)^2 + (z-z_j)^2} + cdt - cdt + \epsilon_{i} - \epsilon_{j}
$$
The clock bias is eliminated. The single differenced pseudorange equation for the $i^{th}$ and $j^{th}$ satellite is given as:
$$
\rho_{ij} = \sqrt{(x-x_i)^2 + (y-y_i)^2 + (z-z_i)^2} - \sqrt{(x-x_j)^2 + (y-y_j)^2 + (z-z_j)^2} + \epsilon_{ij}
$$
where $\epsilon_{ij} = \epsilon_{i} - \epsilon_{j}$ is the remaining noise in the single differenced pseudorange.

We will feed the KalmanNet with the single differenced pseudorange measurements and the KalmanNet will output the position and velocity of the reciever in the ECEF frame. This requires that at least 4 satellites are in view of the reciever.